In [ ]:
if 'NOTEBOOK_INITIATED_FLAG' not in globals():
    NOTEBOOK_INITIATED_FLAG = True
    %cd ..
# %pwd

from feature_engineering import feature_eng_steps

c:\projects\aws


In [12]:
import json
import sys
import time
import pickle
from collections import Counter

import pandas as pd
import requests

url = "https://axieinfinity.com/graphql-server-v2/graphql"

Query

In [8]:
query = """
query GetAxieBriefList($auctionType: AuctionType, $criteria: AxieSearchCriteria, $from: Int, $sort: SortBy, $size: Int, $owner: String) {
  axies(auctionType: $auctionType, criteria: $criteria, from: $from, sort: $sort, size: $size, owner: $owner) {
    results {
      ...AxieBrief
    }
  }
}

fragment AxieBrief on Axie {
  id
  class
  breedCount
  image
  auction {
    currentPrice
    currentPriceUSD
  }
  battleInfo {
    banned
  }
  parts {
    name
    class
    type
    specialGenes
  }
}
"""
def variables(fromm):
    return {
        "from": fromm,
        "size": 5,
        "sort": "PriceAsc",
        "auctionType": "Sale",
        "owner": None,
        "criteria": {
            "region": None,
            "parts": None,
            "bodyShapes": None,
            "classes": None,
            "stages": None,
            "numMystic": None,
            "pureness": None,
            "title": None,
            "breedable": None,
            "breedCount": None,
            "hp": [],
            "skill": [],
            "speed": [],
            "morale": [],
        },
    }

Filter to apply

In [9]:
with open('./model_training/_RF.pkl', 'rb') as f: # RF, KNN, , tree, polynomial, SVM
    model = pickle.load(f)

Do the request and measure performance

In [18]:
start = time.time()
request = requests.post(
            url,
            json={"query": query, "variables": variables(0)},
            verify=False,
        )
end = time.time()
print("Time elapsed request: ", end - start, "s")
# Check the data retrieved
if request.status_code == 200:
    jsn_data = request.json()
else:
    raise Exception(f"Unexpected status code returned: {request.status_code}")
jsn_data = request.json()["data"]["axies"]["results"]
jsn = []
n_found = 0
not_parsed_axies = 0
if jsn_data:
    for ax in jsn_data:
        # print(json.dumps(ax, indent=4, sort_keys=True))
        if ax["battleInfo"]["banned"] == False:
            try:
                id_ = ax["id"]
                class_ = ax["class"]
                price = ax["auction"]["currentPriceUSD"]
                breedCount = ax["breedCount"]
                pureness = Counter(
                    [
                        ax["parts"][0]["class"],
                        ax["parts"][1]["class"],
                        ax["parts"][2]["class"],
                        ax["parts"][3]["class"],
                        ax["parts"][4]["class"],
                        ax["parts"][5]["class"],
                    ]
                ).most_common(1)[0][1]
                eyes = ax["parts"][0]["name"]
                ears = ax["parts"][1]["name"]
                back = ax["parts"][2]["name"]
                mouth = ax["parts"][3]["name"]
                horn = ax["parts"][4]["name"]
                tail = ax["parts"][5]["name"]
                axie_info = pd.DataFrame({
                    "Id": id_,
                    "BreedCount": breedCount,
                    "Pureness": pureness,
                    "Class": class_,
                    "Eyes": eyes,
                    "Ears": ears,
                    "Back": back,
                    "Mouth": mouth,
                    "Horn": horn,
                    "Tail": tail,
                    "Price": price,
                }, index=[0])
                start = time.time()
                axie_values = feature_eng_steps(axie_info)
                end = time.time()
                print("Time elapsed in feature engineering per axie: ", end - start, "s")
                start = time.time()
                price_prediction = model.predict(axie_values.reshape(1,-1))[0]
                end = time.time()
                print("Time elapsed in inference per axie: ", end - start, "s")
            except:
                not_parsed_axies += 1
            else:
                if ( 
                    ( price + 100 <  model.predict(axie_values.reshape(1,-1))[0] )
                    and (price > 50)
                    ) :
                    print('Buy. Price: ',price,' Price prediction: ',price_prediction)
                    n_found += 1
                else:
                    print('Dont buy. Price: ',price,' Price prediction: ',price_prediction)


jsn = json.dumps(jsn, indent=2)
print("Estimated size: " + str(sys.getsizeof(jsn) / 1024) + "KB")
print(f"Axies failed to  parse: ",not_parsed_axies,"/",len(jsn_data) )
print(f"Amount of bargains: {n_found}")
# print(jsn)


c:\projects\aws\venv\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'axieinfinity.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Time elapsed request:  2.0906362533569336 s
Estimated size: 0.0498046875KB
Axies failed to  parse:  100 / 100
Amount of bargains: 0
